In [1]:
#Imports
import numpy
import tensorflow
from sklearn import preprocessing, model_selection
from lottery_ticket_hypothesis import PrunableDense
from tensorflow.keras import models, layers, activations
from tensorflow import optimizers, initializers, losses, metrics

In [2]:
#Tries to enable dynamic memory allocation on GPUs
try:
    for i in tensorflow.config.experimental.list_physical_devices("GPU"):
        tensorflow.config.experimental.set_memory_growth(i, True)
except:
    print("Dynamic memory allocation failed on GPU device " + i)

In [3]:
def create_neural_network_prunable():
    """Prunable model of a fully-connected multilayer perceptron"""
    net = models.Sequential()
    net.add(PrunableDense(256, activation=activations.softsign, name="Dense0", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(128, activation=activations.softsign, name="Dense1", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(64, activation=activations.softsign, name="Dense2", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(32, activation=activations.softsign, name="Dense3", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(1, activation=activations.tanh, name="Output", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    return net

In [4]:
#Data reading and train/test separation
X = numpy.loadtxt("poco_1.prn", skiprows=11, usecols=(1, 2, 3, 4, 5, 6, 7), dtype=numpy.float32)
y_str = numpy.loadtxt("poco_1.prn", skiprows=11, usecols=8, dtype=numpy.str)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(list(set(y_str)))
y = label_encoder.transform(y_str)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
#Building, saving initial values of kernel/bias, compiling and training model
print("Prunable network:")
nn = create_neural_network_prunable()
nn.build(x_train.shape)
for i in nn.layers:
    i.save_kernel()
    i.save_bias()
nn.summary()
nn.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
print("Before pruning:")
nn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_train, y_train))
loss, accuracy = nn.evaluate(x_test, y_test, verbose=0)
print("Loss:", loss)
print("Accuracy:", accuracy)

Prunable network:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense0 (PrunableDense)       multiple                  2048      
_________________________________________________________________
Dense1 (PrunableDense)       multiple                  32896     
_________________________________________________________________
Dense2 (PrunableDense)       multiple                  8256      
_________________________________________________________________
Dense3 (PrunableDense)       multiple                  2080      
_________________________________________________________________
Output (PrunableDense)       multiple                  33        
Total params: 45,313
Trainable params: 45,313
Non-trainable params: 0
_________________________________________________________________
Before pruning:
Train on 1134 samples, validate on 1134 samples
Epoch 1/10
1134/1134 [==============

In [6]:
#Creating list of weights, saving weight value and index
#TODO optimize?
l = []
for i in range(len(nn.layers)):
    t1, t2 = nn.layers[i].kernel, nn.layers[i].bias
    for j in range(t1.shape[0]):
        for k in range(t1[j].shape[0]):
            l.append((t1[j][k].numpy(), i, j, k))
    for j in range(t2.shape[0]):
        l.append((t2[j].numpy(), i, j))

In [7]:
#Sorting weights for pruning
#TODO change p to p**(1/n)
s = sorted(l, key=lambda x: x[0])
del l
p = int(numpy.floor((9. / 10.) * len(s)))
s = s[:p]

In [8]:
#Creating dictionaries for separating weights and bias by layer
to_prune_dict_kernel, to_prune_dict_bias = {}, {}
for i in range(len(nn.layers)):
    to_prune_dict_kernel[i] = []
    to_prune_dict_bias[i] = []
for i in s:
    if len(i) > 3:
        to_prune_dict_kernel[i[1]].append(i[2:])
    else:
        to_prune_dict_bias[i[1]].append(i[2])
del s

In [9]:
#Creating pruning Tensor for pruning weights and pruning each layer kernel
for i in to_prune_dict_kernel.keys():
    if not to_prune_dict_kernel[i]:
        continue
    v = tensorflow.Variable(tensorflow.ones(nn.layers[i].kernel.shape))
    u = tensorflow.Variable(tensorflow.zeros(len(to_prune_dict_kernel[i])))
    t = tensorflow.tensor_scatter_nd_update(v, to_prune_dict_kernel[i], u)
    if tensorflow.math.reduce_any(t == 0):
        nn.layers[i].prune_kernel(t)

In [10]:
#Creating pruning Tensor for bias and pruning each layer
for i in to_prune_dict_bias.keys():
    if not to_prune_dict_bias[i]:
        continue
    v = tensorflow.Variable(tensorflow.ones(nn.layers[i].bias.shape))
    u = tensorflow.Variable(tensorflow.zeros(len(to_prune_dict_bias[i])))
    t = tensorflow.tensor_scatter_nd_update(v, to_prune_dict_bias[i], u)
    if tensorflow.math.reduce_any(t == 0):
        nn.layers[i].prune_bias(t)
del to_prune_dict_bias, to_prune_dict_kernel

In [11]:
#Restoring initial values
for i in nn.layers:
    i.restore_kernel()
    i.restore_bias()

In [12]:
#Training again
print("After pruning:")
nn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_train, y_train))
loss, accuracy = nn.evaluate(x_test, y_test, verbose=0)
print("Loss:", loss)
print("Accuracy:", accuracy)

After pruning:
Train on 1134 samples, validate on 1134 samples
Epoch 1/10
1134/1134 [==============================] - 0s 106us/sample - loss: 5.1040 - binary_accuracy: 0.6631 - val_loss: 4.9988 - val_binary_accuracy: 0.6631
Epoch 2/10
1134/1134 [==============================] - 0s 92us/sample - loss: 1.5554 - binary_accuracy: 0.6464 - val_loss: 0.6645 - val_binary_accuracy: 0.5494
Epoch 3/10
1134/1134 [==============================] - 0s 92us/sample - loss: 0.6716 - binary_accuracy: 0.6605 - val_loss: 0.6525 - val_binary_accuracy: 0.7681
Epoch 4/10
1134/1134 [==============================] - 0s 92us/sample - loss: 0.6210 - binary_accuracy: 0.8131 - val_loss: 0.5858 - val_binary_accuracy: 0.6922
Epoch 5/10
1134/1134 [==============================] - 0s 134us/sample - loss: 0.5604 - binary_accuracy: 0.6614 - val_loss: 0.5353 - val_binary_accuracy: 0.6631
Epoch 6/10
1134/1134 [==============================] - 0s 113us/sample - loss: 0.5181 - binary_accuracy: 0.6631 - val_loss: 0.501

In [13]:
#Printing active edges
print("Active edges:")
t = tensorflow.Variable(0, dtype=tensorflow.int64)
for i in nn.layers:
    j = tensorflow.math.count_nonzero(i.trainable_channels)
    t.assign_add(j)
    print(i.name, ":", j)
print("Total:", t.numpy())

Active edges:
Dense0 : tf.Tensor(722, shape=(), dtype=int64)
Dense1 : tf.Tensor(1757, shape=(), dtype=int64)
Dense2 : tf.Tensor(1134, shape=(), dtype=int64)
Dense3 : tf.Tensor(430, shape=(), dtype=int64)
Output : tf.Tensor(8, shape=(), dtype=int64)
Total: 4051


In [14]:
t.assign(0)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=0>

In [15]:
print("Disabled edges:")
for i in nn.layers:
	j = tensorflow.reduce_sum(tensorflow.cast((i.trainable_channels == 0), dtype=tensorflow.int64))
	t.assign_add(j)
	print(i.name, ":", j)
print("Total:", t.numpy())

Disabled edges:
Dense0 : tf.Tensor(1070, shape=(), dtype=int64)
Dense1 : tf.Tensor(31011, shape=(), dtype=int64)
Dense2 : tf.Tensor(7058, shape=(), dtype=int64)
Dense3 : tf.Tensor(1618, shape=(), dtype=int64)
Output : tf.Tensor(24, shape=(), dtype=int64)
Total: 40781
